# Magic Hands
---

In this project, we are going to work on our first Deep Learning project where we are going to train a Deep Neural Network to recognize your hand patterns. This can be used to create a simple game or control system where based on the detected pattern, you can execute a certain script that will perform some gesture-specific action.

We are using the Tensorflow framework for this project.

## Importing Project Dependencies
---

As the first step of this project, we are going to import all the necessary project dependencies.

In [1]:
# perform OS related operations
import os

# dependencies for model training and data handling
import numpy as np
import pandas as pd
import tensorflow as tf
import tensorflow.keras as keras

# dependencies for working with images
import PIL
import matplotlib.image as mpimg
import matplotlib.pyplot as plt

## Preprocessing Data
---

Now, if you look at the images in the dataset, you will notice two things.
* The dimensions for the images are different.
* The images are rectangular in shape.

Working with consistent data is easier than working with data that is inconsistent. We need to make all the items within our dataset consistent, because our model is going to expect an input of the same size and not varying size.

So for the preprocessing, we are going to perform a center crop on all the images within the dataset, and make all the images of the same dimension.

First, we will perform the center crop on all the images to make them square.

In [2]:
# getting data directories for different labels

data_dir = os.path.join(os.getcwd(), "data")
label_dir = [os.path.join(data_dir, label) for label in os.listdir(data_dir) if not label.startswith('.')]
label_dir

['/Users/amansharma2910/projects/magic_hands/data/0',
 '/Users/amansharma2910/projects/magic_hands/data/1',
 '/Users/amansharma2910/projects/magic_hands/data/2']

In [3]:
# defining function to square center crop all the images in a directory

def square_crop_images(dir: str) -> None:
    images = [os.path.join(dir, image) for image in os.listdir(dir) if (image.endswith(".jpg") and not image.startswith("."))]
    for img in images:
        im = PIL.Image.open(img)
        
        width, height = im.size
        size = min(width, height)
        
        left = (width - size)/2
        top = (height - size)/2
        right = (width + size)/2
        bottom = (height + size)/2

        # crop the center of the image
        im = im.crop((left, top, right, bottom))

        # save image
        im.save(img)

In [4]:
# performing center crop on the dataset images

for dir in label_dir:
    square_crop_images(dir)

Now, we have square cropped the images. We need not resize them now. The images will be automatically resized as we load them into a Tensorflow compatible dataset.

## Loading Dataset
---

The next step is loading our dataset into a Tensorflow compatible dataset format, so we can finally begin training our deep learning model.

For this, we are going to use the `image_dataset_from_directory` util method defined provided by Tensorflow. You can refer to the docs for the same here: https://www.tensorflow.org/api_docs/python/tf/keras/utils/image_dataset_from_directory 

In [6]:
img_size = 256
batch_size = 8

In [24]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  label_mode="int",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size)

Found 600 files belonging to 3 classes.
Using 480 files for training.


In [25]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  label_mode="int",
  seed=123,
  image_size=(img_size, img_size),
  batch_size=batch_size)

Found 600 files belonging to 3 classes.
Using 120 files for validation.


We have created the datasets. What this function basically does is, it converts each image into a tensor as it loads it, and associates a label with it. Let us verify that the dataset is working as intended.

In [26]:
for batch in train_ds:
    images, labels = batch
    print(images.shape)
    print(labels.shape)
    break

(8, 256, 256, 3)
(8,)


The following are the class names respectively for the images in the dataset.

In [28]:
train_ds.class_names

['0', '1', '2']

With this, our images are loaded. Now let us visualize our dataset.

## Visualizing the Data
---